# Optimization Methods Project

In [2]:
import gurobipy as gp
import pandas as pd
import numpy as np

## Question 2

### Load Data

In [3]:
df_city_sm = pd.read_csv("files/cities_small.csv")
df_city_md = pd.read_csv("files/cities_med.csv")
df_city_lg = pd.read_csv("files/cities_large.csv")
df_pkgs_sm = pd.read_csv("files/packages_small.csv")
df_pkgs_md = pd.read_csv("files/packages_med.csv")
df_pkgs_lg = pd.read_csv("files/packages_large.csv")

In [4]:
df_city_sm.head()

,id,lat,lon
0,0,37.618806,-122.375417
1,1,36.080044,-115.152235
2,2,33.636700,-84.427864
3,3,39.861667,-104.673167
4,4,33.942496,-118.408049


In [5]:
cities = df_city_sm.id.to_list()

In [6]:
cities

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [13]:
def compute_dist(city1_row, city2_row):
    city1_lat = city1_row.lat.item()
    city1_lon = city1_row.lon.item()
    city2_lat = city2_row.lat.item()
    city2_lon = city2_row.lon.item()
    return np.sqrt((city1_lat - city2_lat)**2 + (city1_lon - city2_lon)**2)

In [11]:
df_city_sm[df_city_sm.id == 0]

,id,lat,lon
0,0,37.618806,-122.375417


In [14]:
df_city_sm[df_city_sm.id == 0].lat.item()

37.6188055555556

In [26]:
dists = {}
for i, city1 in enumerate(cities):
    city1_row = df_city_sm[df_city_sm.id == city1]
    for city2 in cities[i:]:
        city2_row = df_city_sm[df_city_sm.id == city2]
        if (city1, city2) not in dists:
            curr_dist = compute_dist(city1_row, city2_row)
            dists[(city1, city2)] = curr_dist
            dists[(city2, city1)] = curr_dist

In [27]:
dists

{(0, 0): 0.0,
 (0, 1): 7.385265389230029,
 (1, 0): 7.385265389230029,
 (0, 2): 38.15591593897656,
 (2, 0): 38.15591593897656,
 (0, 3): 17.84376868899152,
 (3, 0): 17.84376868899152,
 (0, 4): 5.408813028490058,
 (4, 0): 5.408813028490058,
 (0, 5): 25.773889892811205,
 (5, 0): 25.773889892811205,
 (0, 6): 41.49611653074913,
 (6, 0): 41.49611653074913,
 (0, 7): 48.60553382145602,
 (7, 0): 48.60553382145602,
 (0, 8): 39.29147706731318,
 (8, 0): 39.29147706731318,
 (0, 9): 42.082014644191126,
 (9, 0): 42.082014644191126,
 (0, 10): 34.74170158291984,
 (10, 0): 34.74170158291984,
 (0, 11): 10.870017183439154,
 (11, 0): 10.870017183439154,
 (0, 12): 28.091268550605463,
 (12, 0): 28.091268550605463,
 (0, 13): 48.30461965636616,
 (13, 0): 48.30461965636616,
 (0, 14): 30.044770767831277,
 (14, 0): 30.044770767831277,
 (0, 15): 51.58763290393545,
 (15, 0): 51.58763290393545,
 (0, 16): 11.176730921164692,
 (16, 0): 11.176730921164692,
 (1, 1): 0.0,
 (1, 2): 30.821370630396732,
 (2, 1): 30.821370630

In [19]:
N = len(df_city_sm.index)

In [24]:
dists = np.zeros((N, N))
for i, city1 in enumerate(cities):
    city1_row = df_city_sm[df_city_sm.id == city1]
    for city2 in cities[i:]:
        city2_row = df_city_sm[df_city_sm.id == city2]
        curr_dist = compute_dist(city1_row, city2_row)
        dists[city1, city2] = curr_dist
        dists[city2, city1] = curr_dist

In [25]:
dists

array([[ 0.        ,  7.38526539, 38.15591594, 17.84376869,  5.40881303,
        25.77388989, 41.49611653, 48.60553382, 39.29147707, 42.08201464,
        34.74170158, 10.87001718, 28.09126855, 48.30461966, 30.04477077,
        51.5876329 , 11.17673092],
       [ 7.38526539,  0.        , 30.82137063, 11.14053581,  3.89479568,
        18.39203208, 34.2141481 , 41.54601149, 32.3847639 , 34.69721975,
        27.87496286,  5.67853499, 20.72737158, 41.24228131, 23.63088835,
        44.59070019,  4.10655202],
       [38.15591594, 30.82137063,  0.        , 21.18071044, 33.98156056,
        12.63149404,  3.81958062, 12.74365816,  8.64279447,  6.06987624,
         9.03725639, 28.46303265, 11.50848515, 12.45129519, 14.27547112,
        16.00885202, 27.58446205],
       [17.84376869, 11.14053581, 21.18071044,  0.        , 14.95605425,
        10.33459082, 24.17512311, 30.81416096, 21.44898787, 26.01117566,
        16.89793406,  7.36315835, 13.58825875, 30.51579171, 12.50351052,
        33.7595663 

In [28]:
df_pkgs_sm.head()

,origin,destination,packages
0,2,15,2878
1,2,6,3745
2,2,3,2973
3,2,5,4868
4,2,8,2796


In [31]:
pkgs = np.zeros((N, N))
for i, row in df_pkgs_sm.iterrows():
    pkgs[row.origin, row.destination] = row.packages

In [32]:
pkgs

array([[0.000e+00, 3.578e+03, 1.464e+03, 2.363e+03, 7.187e+03, 1.729e+03,
        7.070e+02, 0.000e+00, 5.710e+02, 1.820e+02, 2.851e+03, 2.150e+03,
        1.322e+03, 1.497e+03, 9.880e+02, 1.379e+03, 2.366e+03],
       [3.584e+03, 0.000e+00, 2.417e+03, 4.179e+03, 6.083e+03, 2.520e+03,
        8.890e+02, 0.000e+00, 1.056e+03, 6.480e+02, 2.709e+03, 2.694e+03,
        1.378e+03, 1.185e+03, 1.411e+03, 7.830e+02, 5.759e+03],
       [1.645e+03, 2.423e+03, 0.000e+00, 2.973e+03, 2.764e+03, 4.868e+03,
        3.745e+03, 5.517e+03, 2.796e+03, 4.772e+03, 3.898e+03, 1.509e+03,
        2.782e+03, 3.910e+03, 2.598e+03, 2.878e+03, 2.532e+03],
       [2.649e+03, 4.168e+03, 3.078e+03, 0.000e+00, 4.378e+03, 4.231e+03,
        1.067e+03, 1.630e+03, 1.444e+03, 1.649e+03, 3.111e+03, 4.385e+03,
        2.313e+03, 1.074e+03, 2.459e+03, 1.113e+03, 4.164e+03],
       [6.995e+03, 6.167e+03, 2.762e+03, 4.439e+03, 0.000e+00, 3.293e+03,
        7.270e+02, 0.000e+00, 1.017e+03, 9.070e+02, 3.716e+03, 2.616e+03,
    

In [33]:
pkgs = {}
for i in range(N):
    pkgs[(i, i)] = 0

for i, row in df_pkgs_sm.iterrows():
    pkgs[(row.origin, row.destination)] = row.packages

In [34]:
pkgs

{(0, 0): 0,
 (1, 1): 0,
 (2, 2): 0,
 (3, 3): 0,
 (4, 4): 0,
 (5, 5): 0,
 (6, 6): 0,
 (7, 7): 0,
 (8, 8): 0,
 (9, 9): 0,
 (10, 10): 0,
 (11, 11): 0,
 (12, 12): 0,
 (13, 13): 0,
 (14, 14): 0,
 (15, 15): 0,
 (16, 16): 0,
 (2, 15): 2878,
 (2, 6): 3745,
 (2, 3): 2973,
 (2, 5): 4868,
 (2, 8): 2796,
 (2, 13): 3910,
 (2, 12): 2782,
 (2, 1): 2423,
 (2, 4): 2764,
 (2, 7): 5517,
 (2, 9): 4772,
 (2, 14): 2598,
 (2, 10): 3898,
 (2, 16): 2532,
 (2, 0): 1645,
 (2, 11): 1509,
 (15, 2): 2877,
 (15, 6): 1761,
 (15, 3): 1078,
 (15, 5): 1488,
 (15, 8): 1218,
 (15, 13): 1928,
 (15, 12): 891,
 (15, 1): 782,
 (15, 4): 1270,
 (15, 7): 6121,
 (15, 9): 2020,
 (15, 14): 777,
 (15, 10): 3475,
 (15, 16): 539,
 (15, 0): 1379,
 (15, 11): 363,
 (6, 2): 3750,
 (6, 15): 1756,
 (6, 3): 1067,
 (6, 5): 1843,
 (6, 8): 1436,
 (6, 13): 2740,
 (6, 12): 1758,
 (6, 1): 891,
 (6, 4): 725,
 (6, 7): 2971,
 (6, 9): 1764,
 (6, 14): 862,
 (6, 10): 2797,
 (6, 16): 1099,
 (6, 0): 705,
 (6, 11): 156,
 (3, 2): 3078,
 (3, 15): 1113,
 (3, 